В финальном задании будем анализировать таблицу с поездками на байках

In [1]:
import pandas as pd

data = pd.read_csv ('Data/citibike.csv')
data_bike = data.copy()

#data_bike.info()

#информация по признакам
# starttime — время начала поездки (дата, время);
# stoptime — время окончания поездки (дата, время);
# start station id — идентификатор стартовой стоянки(можно в категорию перевести);
# start station name — название стартовой стоянки(можно в категорию перевести);
# start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;
# end station id — идентификатор конечной стоянки(можно в категорию перевести);
# end station name — название конечной стоянки (можно в категорию перевести);
# end station latitude, end station longitude — географическая широта и долгота конечной стоянки;
# bikeid — идентификатор велосипеда;
# usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда) (можно в категорию перевести);
# birth year — год рождения клиента;
# gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина) (можно в категорию перевести)


# Задание 1 Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.
#display(data_bike.head(n=2))
data_bike['start station id'].value_counts()

# Задание 2 Велосипед с каким идентификатором является самым популярным?
data_bike['bikeid'].value_counts()

# Задание 3 Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? 
# В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. Ответ округлите до сотых
user = data_bike['usertype'].value_counts(normalize=True)
#display(user)

# Задание 4 Кто больше совершает велопоездок мужчины или женщины, запишете число
user_gender = data_bike['gender'].value_counts(normalize=False)
#display(user_gender.max())

# Задание 5 удалить дублирующие друг друга признаки
data_bike = data_bike.drop(['start station id', 'end station id'], axis=1)		

# Задание 6 Замените признак birth year на более понятный признак возраста клиента age. 
# Годом отсчёта возраста выберите 2018 год. Столбец birth year удалите из таблицы. Сколько поездок совершено клиентами старше 60 лет
data_bike['age'] = 2018 - data_bike['birth year']
data_bike = data_bike.drop('birth year', axis=1)
data_bike[data_bike['age'] > 60].shape

# Задание 7 Создайте признак длительности поездки trip duration. 
# Для этого вычислите интервал времени между временем окончания поездки (stoptime) и её началом (starttime). Сколько целых минут длилась поездка под индексом 3 в таблице?

#создадим новый признак
data_bike['starttime'] = pd.to_datetime(data_bike['starttime'])
data_bike['stoptime'] = pd.to_datetime(data_bike['stoptime'])
data_bike['trip duration'] = data_bike['stoptime'] - data_bike['starttime']
#data_bike.head(n=4) 

# Задание 8 Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день (суббота или воскресенье), и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.
data_bike['weekend'] = data_bike['starttime'].dt.dayofweek
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0

data_bike['weekend'] = data_bike['weekend'].apply(get_weekend) 
display(data_bike['weekend'].value_counts()) 
    
# Задание 9 Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки. Условимся, что:
# поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;
# поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;
# поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;
# поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).
# Во сколько раз количество поездок, совершённых днём, больше, чем количество поездок, совёршенных ночью, за представленный в данных период времени? Ответ округлите до целых.

#функция которая смотри сколько часов и возвращает значение день или утро в зависимости от часа, когда началась поездка
def searh_night_day (hours):
    time = hours.hour
    if 0 <= time <= 6: 
        return 'night'
    if 6 < time <= 12:
        return 'morning'
    if 12 < time <= 18:
        return 'day'
    if 18 < time <= 23:
        return 'evening'
#далее делаем новый столбец и применим функцию ко всем строкам
data_bike['time_of_day'] = data_bike['starttime'].apply(searh_night_day)

#сделаем выборку по маске и посмотри кол строк:
day = data_bike[data_bike['time_of_day'] == 'day'].shape[0]
night = data_bike[data_bike['time_of_day'] == 'night'].shape[0]
print(f'Количество поездок днем = {day}, кол поездок ночью = {night}, те в {round(day/night)} раз больше')
 

weekend
0    184865
1    115135
Name: count, dtype: int64

Количество поездок днем = 143012, кол поездок ночью = 15085, те в 9 раз больше
